In [ ]:
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━❮Bibliotecas❯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
from scipy.stats import ks_2samp
from collections import OrderedDict
from operator import itemgetter
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, KFold
from sklearn.metrics import confusion_matrix, recall_score, precision_score, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from joblib import dump
#━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━❮◆❯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Ler o arquivo Excel
df = pd.read_excel('/content/dataset.xlsx')

# Filtrar casos positivos e negativos
df_positive = df[df['SARS-Cov-2 exam result'] == 'positive']
df_negative = df[df['SARS-Cov-2 exam result'] == 'negative']

# Remover colunas específicas
df = df.drop(['Patient addmited to regular ward (1=yes, 0=no)',
              'Patient addmited to semi-intensive unit (1=yes, 0=no)',
              'Patient addmited to intensive care unit (1=yes, 0=no)'], axis=1)

# Definir 'Patient ID' como índice do DataFrame
df = df.set_index('Patient ID')


In [ ]:
nan_analyze = df.isna().sum()/len(df)

print("Quantidade de tuplas:", len(df))
print("Percentual médio missing values:", round(nan_analyze.mean()*100,1),"%")

label_perc = []
for i in np.arange(0, len(df.columns), 10):
    label_perc.append(str(i)+"%")
plt.figure(figsize=[10,40])

plt.yticks(np.arange(len(df.columns)), nan_analyze.index.values)
plt.xticks(np.arange(0, 1.1, .1), label_perc)

plt.ylim(0,len(df.columns))

plt.barh(np.arange(len(df.columns)), nan_analyze)

In [ ]:
df_filtered = df[~np.isnan(df['Hematocrit'])]
nan_analyze_filtered = df_filtered.isna().sum()/len(df_filtered)

print("Quantidade de tuplas:", len(df_filtered))
print("Percentual médio missing values:", round(nan_analyze_filtered.mean()*100,1),"%")


label_perc = []
for i in np.arange(0, 110, 10):
    label_perc.append(str(i)+"%")
plt.figure(figsize=[10,40])

plt.yticks(np.arange(len(df_filtered.columns)), nan_analyze_filtered.index.values)
plt.xticks(np.arange(0, 1.1, .1), label_perc)

plt.ylim(0,len(df_filtered.columns))

plt.barh(np.arange(len(df_filtered.columns)), nan_analyze_filtered)

In [ ]:
features = df_filtered.dtypes[df_filtered.dtypes=='float64'].index.values

ks_list = []
pvalue_list = []
feature_list = []

for feature in features:
    
    positive = df_positive[~np.isnan(df_positive[feature])]
    negative = df_negative[~np.isnan(df_negative[feature])]
    
    if len(positive)*len(negative)>0:
        ks, pvalue = ks_2samp(positive[feature], negative[feature])
        ks_list.append(ks)
        pvalue_list.append(pvalue)
        feature_list.append(feature)
        
df_ks = pd.DataFrame(data=zip(ks_list,pvalue_list),columns=['ks', 'pvalue'],index=feature_list)
df_ks = df_ks.sort_values(by='ks',ascending=True)

df_ks['ks']
plt.figure(figsize=(8,15))
plt.yticks(np.arange(len(df_ks)), df_ks.index.values)
plt.title('Diferença entre positivo VS negativo')
plt.barh(np.arange(len(df_ks)), df_ks['ks'])

In [ ]:
df_treated = df_filtered
cat_features = df_filtered.dtypes[df_filtered.dtypes == 'object'].index.values

for feature in cat_features:
    df_treated[feature] = df_treated[feature].fillna(df_treated[feature].mode().values[0]) 
    
df_treated = df_treated.fillna(df_treated.median())

df_treated_dummies = pd.get_dummies(df_treated, drop_first=True, dtype='bool')

columns = list(df_treated_dummies.drop(labels=['SARS-Cov-2 exam result_positive'],axis=1).columns.values)
columns.append('SARS-Cov-2 exam result_positive')

df_treated_dummies = df_treated_dummies[columns]

In [ ]:
ax = df_treated['SARS-Cov-2 exam result'].value_counts().plot(kind='bar',
                                    figsize=(14,8))
ax.set_xticklabels(['negative', 'positive'], rotation=0, fontsize=20)

In [ ]:
# Removendo colunas que contêm apenas valores NaN
df_treated_dummies = df_treated_dummies.dropna(axis=1, how='all')

# Verificar o número de colunas antes da imputação
print("Número de colunas antes da imputação:", df_treated_dummies.iloc[:,:-1].shape[1])

# Imputação
imputer = SimpleImputer(strategy='mean')  # Ou use 'median' se preferir
x_imputed = imputer.fit_transform(df_treated_dummies.iloc[:,:-1])

# Verificar o número de colunas após a imputação
print("Número de colunas após a imputação:", x_imputed.shape[1])

# Criar DataFrame após a imputação
x = pd.DataFrame(x_imputed, columns=df_treated_dummies.iloc[:,:-1].columns)

# Adicionando a coluna 'Random'
x['Random'] = np.random.rand(x.shape[0])

# Divisão em conjuntos de treino e teste
y = df_treated_dummies['SARS-Cov-2 exam result_positive']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

# Aplicar SMOTE
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [ ]:
kfold = KFold(n_splits=20)
kfold = KFold(n_splits=20, shuffle=True, random_state=42)


param_grid = {
    'min_samples_split':[2, 4, 6],
    'min_samples_leaf':[2, 4, 6],
    'n_estimators':[10, 30, 50],
    'max_depth':[3, 5]
    }

clf_rf = RandomForestClassifier(random_state=42)
grid = GridSearchCV(estimator=clf_rf, param_grid=param_grid, cv=kfold, scoring='recall', n_jobs=-1)
grid.fit(X=X_train, y=y_train)
clf_rf = grid.best_estimator_

In [ ]:
# Treinando o modelo com o conjunto de dados balanceado por SMOTE
clf_rf.fit(X_train_resampled, y_train_resampled)

# Obtendo a importância das características
cols = x.columns  # Assegure-se de que 'x' contém todas as colunas usadas no modelo
feature_importance = pd.DataFrame(data=clf_rf.feature_importances_, index=cols, columns=['FI'])
feature_importance = feature_importance.sort_values(by='FI', ascending=True)

# Visualização da importância das características
plt.figure(figsize=(8,10))
plt.yticks(np.arange(len(feature_importance)), feature_importance.index.values)
plt.barh(np.arange(len(feature_importance)), feature_importance['FI'])
plt.show()


In [ ]:
# Se necessário, adicione a coluna 'Random' a X_train
X_train['Random'] = np.random.rand(X_train.shape[0])

# Treinar o modelo com X_train que inclui a coluna 'Random'
clf_rf.fit(X_train, y_train)

# Adicionar a coluna 'Random' a X_test
X_test['Random'] = np.random.rand(X_test.shape[0])

# Gerar a matriz de confusão
cm = confusion_matrix(y_test, clf_rf.predict(X_test))

# Plotar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
plt.title('Matriz de Confusão')
plt.xlabel('Previsto')
plt.ylabel('Verdadeiro')
plt.show()

# Calculando e imprimindo o recall
print("Recall treino:", recall_score(y_train, clf_rf.predict(X_train)))
print("Recall validação:", cross_val_score(clf_rf, X_train, y_train, cv=20, scoring='recall', n_jobs=-1).mean())
print("Recall teste:", recall_score(y_test, clf_rf.predict(X_test)))



a

In [ ]:
feature_importance = pd.DataFrame(data=clf_rf.feature_importances_, index=cols, columns=['FI'])
feature_importance = feature_importance.sort_values(by='FI', ascending=True)


# Selecionando as 5 características mais importantes
important_features = feature_importance.sort_values(by='FI', ascending=False).head(5).index.tolist()
print("Top 5 características importantes:", important_features)


In [ ]:
# Suponha que X_train é o DataFrame que foi usado para treinar o modelo
all_features = X_train.columns.tolist()
print("Todas as características utilizadas para treinar o modelo:")
print(all_features)



In [ ]:
# Suponha que X_train é o DataFrame que foi usado para treinar o modelo
all_features = X_train.columns.tolist()
print("Todas as características utilizadas para treinar o modelo:")
print(all_features)



In [ ]:
dump(clf_rf, 'RandomForest.joblib')

In [ ]:


def make_prediction(input_features, model, all_features, important_features):
    """
    Faz uma previsão com base nas entradas do usuário.

    :param input_features: Um dicionário com as características importantes e seus valores.
    :param model: O modelo de classificação treinado para fazer a previsão.
    :param all_features: Lista de todas as características usadas para treinar o modelo.
    :param important_features: Lista das características importantes que o usuário deve fornecer.
    :return: A previsão do modelo.
    """

    # Verificar se todas as características importantes foram fornecidas
    for feature in important_features:
        if feature not in input_features:
            raise ValueError(f"Faltando valor para a característica importante: {feature}")

    # Converter as entradas do usuário em um DataFrame
    user_input_df = pd.DataFrame([input_features])

    # Adicionar colunas faltantes e organizar as colunas na mesma ordem do treinamento
    for feature in all_features:
        if feature not in user_input_df.columns:
            user_input_df[feature] = 0  # Preencher com um valor padrão
    user_input_df = user_input_df[all_features]

    # Fazer a previsão
    prediction = model.predict(user_input_df)

    return prediction

# Exemplo de uso da função:
# Suponha que clf_rf seja o seu modelo treinado, all_features a lista completa de características, 
# e important_features as 5 características principais.
# Você chamaria a função assim:
# result = make_prediction(input_features={'Feature1': valor1, 'Feature2': valor2, ...}, 
#                          model=clf_rf, 
#                          all_features=all_features, 
#                          important_features=important_features)
# print("Previsão do modelo:", result)
